# 随机抽样取件

## 策略

1. 根据票件的5种类型，对着五种类型的包裹按比例提取，以parcel_id为主  
2. 最后把五种包裹合并  

In [1]:
import pandas as pd
import pymysql
import numpy as np
from sqlalchemy import create_engine

## 1. 连接数据库，读取包裹信息

In [15]:
# 数据库连接参数
u_name = "root"
pwd  = "root123"
db = "TEST"
host = "localhost"

# 读取数据库表名
read_table = "i_od_parcel_2025v31_mix_test"
# 写入的数据库表名
write_table = 'i_od_parcel_2025v31_mix_pct1_test'

# 创建连接引擎
engine = create_engine('mysql+pymysql://{u_name}:'
                       '{pwd}@{host}/{db}'
                       '?charset=utf8'.format(
    u_name=u_name,
    pwd=pwd,
    db=db,
    host=host))
old_parcel_df = pd.read_sql_table(read_table,con=engine)

c:\python36\lib\site-packages\pymysql\cursors.py:165: Warning: (1366, "Incorrect string value: '\\xD6\\xD0\\xB9\\xFA\\xB1\\xEA...' for column 'VARIABLE_VALUE' at row 480")
  result = self._query(query)


ValueError: Table i_od_parcel_2025v31_plus_mix_pct1 not found

## 2. 抽样预处理

In [ ]:
# 抽样比例 : 抽样除数
sample_rate = {
    '1%':100,
    '10%':10
}

In [ ]:
# 所有irregular类型的parcel_id
irregular_parcel_id_list =
    list(old_parcel_df.loc[old_parcel_df.parcel_type.isin(["irregular"]),["parcel_id"]].parcel_id.unique())

# 所有isb类型的parcel_id
isb_parcel_id_list = 
    list(d_parcel.loc[old_parcel_df.parcel_type.isin(["isb"]),["parcel_id"]].parcel_id.unique())

# 所有nc类型的parcel_id
nc_parcel_id_list = 
    list(d_parcel.loc[old_parcel_df.parcel_type.isin(["nc"]),["parcel_id"]].parcel_id.unique())

# 所有parcel类型的parcel_id
parcle_parcel_id_list = 
    list(d_parcel.loc[old_parcel_df.parcel_type.isin(["parcel"]),["parcel_id"]].parcel_id.unique())

# 所有small类型的parcel_id
small_parcel_id_list = 
    list(d_parcel.loc[old_parcel_df.parcel_type.isin(["small"]),["parcel_id"]].parcel_id.unique())

In [ ]:
old_parcel_df1 = old_parcel_df.loc[:,['small_id','parcel_id','parcel_type']]

In [ ]:
# 长度
len(old_parcel_df1.loc[old_parcel_df1.parcel_type.isin(['small']),'small_id'])

In [ ]:
a = old_parcel_df1.groupby('parcel_type')

In [ ]:
# 计算出各种类型非重复的数量
type_dict = a.parcel_id.nunique().to_dict()   # 查看去重后的数量
type_dict

In [ ]:
# 源数据 parcel_type: [parcel_id]
old_type_dict = {
    'irregular': irregular_small_id_list,
    'isb': isb_small_id_list, 
    'nc': nc_small_id_list, 
    'parcel': parcle_small_id_list,
    'small': small_small_id_list}


irregular_list=[]
isb_list=[]
nc_list=[]
parcel_list=[]
small_list=[]

# 抽样后的 parcel_type：[parcel_id]
sample_type_dict={
    'irregular': irregular_list, 
    'isb': isb_list, 
    'nc': nc_list, 
    'parcel': parcel_list, 
    'small': small_list}

In [ ]:
import random

def sample_random(divisor):
    ''' 
    抽样函数：传入抽样的 divisor 值，返回抽样完成的 sample_type_dict
    '''
    for k,val in type_dict.items():
        
        while True:
            if len(set(old_type_dict[k])) == round(val/divisor):
                break
            parcel_id = random.choice(old_type_dict[k])
            sample_type_dict[k].append(parcel_id)
    return sample_type_dict

## 3. 抽样%1，并存储数据

In [ ]:
sample_type_dict = sample_random(sample_rate['1%'])

In [ ]:
# 连接列表数据
parcel_li = irregular_list + isb_list + nc_list + parcel_list + small_list

In [ ]:
len(parcel_li)

In [ ]:
# 取出抽样的数据，存入新的df中
new_parcel_df = old_parcel_df.loc[old_parcel_df.parcel_id.isin(parcel_li),:]
new_parcel_df.head()

In [ ]:
# 存入数据库中
parcel_1.to_sql(write_table,engine,if_exists='append',chunksize=10000,index=False)